# IPW, stablized IPW

In [1]:
import pickle as pkl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression, Ridge

In [2]:
data = pkl.load(open('./data/data_p.pkl', 'rb'))
data.head(2)

,confidence,index,time,token_path,term_match,word_net,T,Y,propensity,weighted
0,33.6,11,4.0,0.614881,1.000000,1.0,False,18,0.058034,0.061609
1,33.6,12,3.0,0.430476,0.427287,1.0,False,12,0.011190,0.011317


In [8]:
def IPW(df):
    records = df.shape[0]
    t1 = (df[df['T']==1]['Y'] / df[df['T']==1]['propensity']).sum() / records
    t2 = (df[df['T']==0]['Y'] / (1-df[df['T']==0]['propensity'])).sum() / records
    return t1-t2

In [10]:
def stablized_IPW(df):
    T, NT = df[df['T']==1], df[df['T']==0]
    t1 = (1/(1 / T['propensity']).sum()) * (T['Y'] / T['propensity']).sum()
    t2 = (1/(1 / (1-NT['propensity'])).sum()) * (NT['Y'] / (1-NT['propensity'])).sum()
    return t1-t2

In [11]:
print(f"ATE for data: {IPW(data)}")
print(f"Stablized ATE for data: {stablized_IPW(data)}")

ATE for data: 9.11505335544706
Stablized ATE for data: 11.879692210278009
